In [0]:
!pip install vega

In [0]:
!pip install -U altair vega_datasets

Requirement already up-to-date: altair in /usr/local/lib/python3.6/dist-packages (3.1.0)
Requirement already up-to-date: vega_datasets in /usr/local/lib/python3.6/dist-packages (0.7.0)


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import glob
import numpy as np
import librosa
import os
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import pandas as pd
import vega
import altair as alt
import pandas as pd
import scipy.signal
import scipy.fftpack as fft
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [8]:
#File reading
print('File reading...')
all_dirs = []
for root, dirs, files in os.walk('/content/drive/My Drive/Work and Research/MaSC Research/Coding/Test0'):
        for name in files:
            if '.wav' in name:
                filedir = os.path.join(root, name)
                all_dirs.append(filedir)
file_no = len(all_dirs)
print('Number of files: ' + str(file_no))

#Feature Computation
print('Feature Computation...')
all_db = []
all_mean = []
file_names = []

for i in range(file_no):
    if (librosa.get_duration(filename=all_dirs[i]) == 15.):
        file_names.append(all_dirs[i])
        #Progress report
        if (i==file_no-1):
            print('100%')
        elif (i==int(file_no*0.75)):
            print('75%')
        elif (i==int(file_no*0.5)):
            print('50%')
        elif (i==int(file_no*0.25)):
            print('25%')
        #Load file
        y, sr = librosa.core.load(all_dirs[i], duration=10.)
        #Features
        S = librosa.core.stft(y=y)
        S_db = librosa.core.power_to_db(np.abs(S)**2)
        all_mean.append(np.mean(S_db)) 
        
        S_mel = librosa.feature.melspectrogram(y=y, sr=sr)
        all_db.append(S_mel.flatten()) #length: 82688
feature_no = len(all_db[0])      

print('Number of files with a duration of 15 seconds: ' + str(len(file_names)))

X_train, X_test, y_train, y_test = train_test_split(np.asarray(all_db), np.asarray(file_names), test_size=0.1, random_state=42)
print(X_train.shape)

# #Autoencoder (2000, 100, 50, 100, 2000, 82688)
input_db = Input(shape=(feature_no,))
encoded = Dense(2000, activation='relu')(input_db)
encoded = Dense(100, activation='relu')(encoded)
encoded = Dense(50, activation='relu')(encoded)
decoded = Dense(100, activation='relu')(encoded)
decoded = Dense(2000, activation='relu')(decoded)
decoded = Dense(feature_no, activation='relu')(decoded)

autoencoder = Model(input_db, decoded)
encoder = Model(input_db, encoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_split = 0.2,
                validation_data=(X_test, X_test))

encoded_db = encoder.predict(X_train)


#Standardization
scl1 = StandardScaler()
all_db_scaled = scl1.fit_transform(encoded_db)

#TSNE
db_red2 = TSNE(n_components=2).fit_transform(all_db_scaled)

#KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(db_red2)
clusters = kmeans.predict(db_red2)

#x and y
db1 = []
db2 = []
for i in range(len(db_red2)):
    mel1.append(db_red2[i][0])
    mel2.append(db_red2[i][1])

#Visualization

#Clusters
df1 = pd.DataFrame({'x': np.asarray(db1), 'y': np.asarray(db2), 'color': clusters, 'path': np.asarray(y_train), 'filename': np.asarray(y_train)})
chart1 = alt.Chart(df).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()

#Intensity
df2 = pd.DataFrame({'x': np.asarray(db1), 'y': np.asarray(db2), 'color': all_mean, 'path': np.asarray(y_train), 'filename': np.asarray(y_train)})
chart2 = alt.Chart(df).mark_circle(opacity=0.6, size=60).encode(x='x', y='y', color='color:Q', href='path', tooltip=['filename']).interactive()

#Combined
df3 = pd.DataFrame({'x': np.asarray(db1), 'y': np.asarray(db2), 'color': clusters, 'path': np.asarray(y_train), 'filename': np.asarray(y_train)})
chart3 = alt.Chart(df).mark_circle(size=80).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
df4 = pd.DataFrame({'x': np.asarray(db1), 'y': np.asarray(db2), 'color': all_mean, 'path': np.asarray(y_train), 'filename': np.asarray(y_train)})
chart2 = alt.Chart(df).mark_circle(size=30).encode(x='x', y='y', color='color:Q', href='path', tooltip=['filename']).interactive()

display(chart1)
display(chart2)
display(chart3 + chart4)

File reading...
Number of files: 17
Feature Computation...
25%
50%
75%
100%
Number of files with a duration of 15 seconds: 17
(15, 55168)
Train on 15 samples, validate on 2 samples
Epoch 1/50


ResourceExhaustedError: ignored